[Reference](https://medium.com/python-in-plain-english/16-intermediate-level-python-dictionary-tips-tricks-and-shortcuts-1376859e1adc)

# 1. Print All the Key: Value Pairs in a Nested Dictionary

In [1]:
import requests
import json


def recursive_key_values(dictionary):
    """recursive_key_values.
        Print all keys and values anywhere in a dictionary
    Args:
        dictionary: any dictionary
    Returns:
        tuple:
    """
    for key, value in dictionary.items():
        i = 0
        if type(value) is str:
            yield (key, value)
        elif type(value) is dict:
            yield from recursive_key_values(value)
        elif type(value) in (list, tuple, set):
            for seq_item in value:
                yield from recursive_key_values({f"{key}_{str(i)}": seq_item})
                i = i + 1
        else:
            yield (key, str(value))


mydata: dict = {
    "key2": {
        "key2.0": "value1",
        "key2.1":...,
        "key2.2": {
            "key2.2.0": "value2"
        }
    },
    "key1": {
        "key1.0": "value3"
    },
}
mydata["key3"] = {1, 2, 3} # set
mydata["key4"] = [4, 5, 6] # list
mydata["key5"] = (7, 8, 9) # tuple

if __name__ == "__main__":
    for key, value in recursive_key_values(mydata):
        print(f"{key}: {value}")

key2.0: value1
key2.1: Ellipsis
key2.2.0: value2
key1.0: value3
key3_0: 1
key3_1: 2
key3_2: 3
key4_0: 4
key4_1: 5
key4_2: 6
key5_0: 7
key5_1: 8
key5_2: 9


# 2. Sort a Dictionary

In [2]:
"""
Sorting a dictionary to an Iterable of tuples.
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area
"""

countries: dict = {
    "Taiwan": 36193,
    "Canada": 9984670,
    "United States": 9525067,
    "Russia": 17098246,
    "Argentina": 2780400,
    "Zambia": 752612,
    "China": 9596961,
}

assert dict(sorted(countries.items(), key=lambda kv: kv[1], reverse=False)) == {
    "Taiwan": 36193,
    "Zambia": 752612,
    "Argentina": 2780400,
    "United States": 9525067,
    "China": 9596961,
    "Canada": 9984670,
    "Russia": 17098246,
}

assert dict(sorted(countries.items(), key=lambda kv: kv[1], reverse=True)) == {
    "Russia": 17098246,
    "Canada": 9984670,
    "China": 9596961,
    "United States": 9525067,
    "Argentina": 2780400,
    "Zambia": 752612,
    "Taiwan": 36193,
}

assert dict(sorted(countries.items(), key=lambda kv: kv[0], reverse=False)) == {
   "Argentina": 2780400,
   "Canada": 9984670,
   "China": 9596961,
   "Russia": 17098246,
   "Taiwan": 36193,
   "United States": 9525067,
   "Zambia": 752612,
}

assert dict(sorted(countries.items(), key=lambda kv: kv[0], reverse=True)) == {
    "Zambia": 752612,
    "United States": 9525067,
    "Taiwan": 36193,
    "Russia": 17098246,
    "China": 9596961,
    "Canada": 9984670,
    "Argentina": 2780400,
}

# 3. Merge Two Dictionaries

In [3]:
def merge_dictionaries(left: dict, right: dict) -> dict:
    """Merge two dictionaries using a shallow copy."""
    temp: dict = left.copy()
    temp.update(right)
    return temp


countries: dict = {
    "Taiwan": 36193,
    "Canada": 9984670,
    "United States": 9525067,
    "Russia": 17098246,
    "Argentina": 2780400,
    "Zambia": 752612,
    "China": 9596961,
}

cities: dict = {
    "Toronto": ["Canada", 6082000],
    "New York City": ["United States", 18819000],
    "Moscow": ["Russia", 12410000],
    "Buenos Aires": ["Argentina", 14967000],
    "Shanghai": ["China", 25582000],
    "Lusaka": ["Zambia", 1747152],
    "Taipei": ["Taiwan", 2646204],
}

assert merge_dictionaries(countries, cities) == {
    "Taiwan": 36193,
    "Canada": 9984670,
    "United States": 9525067,
    "Russia": 17098246,
    "Argentina": 2780400,
    "Zambia": 752612,
    "China": 9596961,
    "Toronto": ["Canada", 6082000],
    "New York City": ["United States", 18819000],
    "Moscow": ["Russia", 12410000],
    "Buenos Aires": ["Argentina", 14967000],
    "Shanghai": ["China", 25582000],
    "Lusaka": ["Zambia", 1747152],
    "Taipei": ["Taiwan", 2646204],
}

# 4. Make a Dictionary from Two Lists

In [4]:
def dict_from_two_lists(keys: list, values: list) -> dict:
    """
    Args:
        keys: The list of keys for the dictionary
        values: The list of values for the dictionary
    Returns: A dictionary of key:value pairs
    """
    if len(keys) != len(values):
        raise ValueError("Lists must be of same length")

    return dict(zip(keys, values))


assert dict_from_two_lists(["first", "second", "third"],
                           ["primary", "secondary", "tertiary"]) == {
                               "first": "primary",
                               "second": "secondary",
                               "third": "tertiary",
                           }

# 5. Get all Items in a Nested Dictionary by Key

In [5]:
import json
import requests


def get_values_by_key(dictionary: dict, key: str) -> dict:
    """get_values_by_key.
    Args:
        dictionary (dict): dictionary
        key (str): key
    Returns:
        dict:
    """
    if isinstance(dictionary, dict):
        for k, v in dictionary.items():
            if k == key:
                yield v
            elif isinstance(v, dict):
                for result in get_values_by_key(v, key):
                    yield result
            elif type(v) in (list, tuple):
                for d in v:
                    for seq in get_values_by_key(d, key):
                        if type(seq) in (list, tuple):
                            for inner_item in seq:
                                yield inner_item
                        else:
                            yield seq


dict1 = dict(
    json.loads(
        requests.get("http://ergast.com/api/f1/2004/1/results.json").text))

assert "http://en.wikipedia.org/wiki/McLaren" in list(
    get_values_by_key(dict1, "url"))

# 6. Get all Items in a Nested Dictionary by Value



In [6]:
import json
import requests


def get_key_by_value(dictionary: dict, val: str) -> object:
    """get_values_by_key.
    Args:
        dictionary (dict): dictionary
        val (str): value
    Returns:
        dict:
    """
    if isinstance(dictionary, dict):
        for k, v in dictionary.items():
            if val == v:
                yield k
            elif isinstance(v, dict):
                for result in get_key_by_value(v, val):
                    yield result
            elif isinstance(v, list):
                for list_item in v:
                    for result in get_key_by_value(list_item, val):
                        yield result


dict1 = dict(
    json.loads(
        requests.get("http://ergast.com/api/f1/2004/1/results.json").text))

assert "raceName" in list(get_key_by_value(dict1, "Australian Grand Prix"))

# 7. Create a Dictionary using Dictionary Comprehension

In [8]:
asci_uppercase: dict = {i: chr(+i) for i in range(65, 91, 1)}
digits: dict = {i: chr(+i) for i in range(48, 58, 1)}
asci_lowercase: dict = {i: chr(+i) for i in range(97, 123, 1)}
asci_punctuation: dict = {i: chr(+i) for i in range(32, 48, 1)}
asci_punctuation.update({i: chr(+i) for i in range(123, 127, 1)})
asci_punctuation.update({i: chr(+i) for i in range(91, 97, 1)})
asci_punctuation.update({i: chr(+i) for i in range(58, 65, 1)})
asci_extended: dict = {i: chr(+i) for i in range(128, 255, 1)}
asci_system: dict = {i: hex(i) for i in range(0, 32, 1)}

ascii_chars: dict = {}
ascii_chars.update({"asci_punctuation": asci_punctuation})
ascii_chars.update({"asci_lowercase": asci_lowercase})
ascii_chars.update({"asci_uppercase": asci_uppercase})
ascii_chars.update({"digits": digits})
ascii_chars.update({"asci_extended": asci_extended})
ascii_chars.update({"asci_system": asci_system})

print(ascii_chars)

{'asci_punctuation': {32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 123: '{', 124: '|', 125: '}', 126: '~', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@'}, 'asci_lowercase': {97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e', 102: 'f', 103: 'g', 104: 'h', 105: 'i', 106: 'j', 107: 'k', 108: 'l', 109: 'm', 110: 'n', 111: 'o', 112: 'p', 113: 'q', 114: 'r', 115: 's', 116: 't', 117: 'u', 118: 'v', 119: 'w', 120: 'x', 121: 'y', 122: 'z'}, 'asci_uppercase': {65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z'}, 'digits': {48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9'}, 'asci_extended': {128: '\x80', 129: '\x81'

# 8. Use Default Values for New Items

In [9]:
from collections import defaultdict as dd

s = [("John", "Male"), ("John", "48"), ("John", "Married"), ("Jane", "Female"),
     ("Jane", "25")]

dict1: dict = dd(list)

for k, v in s:
    dict1[k].append(v)

assert dict1["John"] == ["Male", "48", "Married"]
assert dict1["Jane"] == ["Female", "25"]

# 9. Convert List of Tuples to Dictionary

In [10]:
from collections import defaultdict as dd

s = [("John", "Male", 25), ("Fred", "Female", 48), ("Sam", "Female", 41),
     ("Jane", "Female", 25)]

dict1 = dd(dict)

for name, gender, age in s:
    dict1[name]["age"] = age
    dict1[name]["gender"] = gender

assert dict1["John"] == {"age": 25, "gender": "Male"}
assert dict1["Fred"] == {"age": 48, "gender": "Female"}
assert dict1["Sam"] == {"age": 41, "gender": "Female"}
assert dict1["Jane"] == {"age": 25, "gender": "Female"}

# 10. Create a Dictionary from a CSV File with Column Headers

In [11]:
from collections import defaultdict as dd
import csv
import requests

url: str = "https://data.london.gov.uk/download/london-borough-profiles/c1693b82-68b1-44ee-beb2-3decf17dc1f8/london-borough-profiles.csv "

boroughs = (requests.get(url).text).split("\n")
reader = csv.DictReader(boroughs, dialect="excel")
dict1 = dd(dict)

for row in reader:
    dict1[row["Code"]] = row

assert dict1["E09000001"]["Area_name"] == "City of London"
assert dict1["E09000032"]["Inner/_Outer_London"] == "Inner London"

# 11. Delete an Item from a Dictionary

In [12]:
digits: dict = {i: chr(+i) for i in range(48, 58, 1)}
key = 50

try:
    val = digits.pop(key)
except KeyError:
    print (f"The item with key {key} did not exist.")
else:
    print(f"Deleted item with key {key} with value {val}.")

Deleted item with key 50 with value 2.


# 12. Make a Deep Copy of a Dictionary

In [13]:
import copy

original = {}
original["object1"] = dict({"a": {"b": [1, 2, 3]}})

dict_shallow = original.copy()
dict_deep = copy.deepcopy(original)

# change the mutable object in original and dict_shallow
original["object1"]["a"]["b"] = [3, 4, 5]

assert id(original["object1"]) == id(dict_shallow["object1"])
assert id(original["object1"]) != id(dict_deep["object1"])

# 13. Reverse Keys and Values

In [14]:
def invert_dictionary(input:dict)->dict:
    return {v: k for k, v in dict1.items()}

dict1 = {"a": 1, "b": 2, "c": 3}

assert invert_dictionary(dict1) == {1: "a", 2: "b", 3: "c"}

# 14. Save Multiple Dictionaries to a File

In [15]:
import shelve

dict1 = {"a": 1, "b": 2, "c": 3}
dict2 = {1: "a", 2: "b", 3: "c"}
dict3 = {}
dict4 = {}

with shelve.open("dictionary-shelf", "c") as shelf:
    shelf["first"] = dict1
    shelf["second"] = dict2

with shelve.open("dictionary-shelf", "r") as shelf:
    dict3 = shelf["first"]
    dict4 = shelf["second"]

assert dict1 == dict3
assert dict2 == dict4
assert dict1 != dict2
assert dict3 != dict4

# 15. Convert from Dictionary to JSON

In [16]:
import json
from datetime import datetime


class PythonObjectEncoder(json.JSONEncoder):

    def default(self, an_object_value):

        if isinstance(an_object_value, str):
            return an_object_value

        try:
            iterable = iter(an_object_value)
            return list(iterable)
        except TypeError:
            pass  # this object is not iterable

        if isinstance(an_object_value, datetime):
            return an_object_value.isoformat()

        elif hasattr(an_object_value, "__class__"):
            return an_object_value.__dict__

        return super().default(an_object_value)


def dict_to_json(the_dict: dict) -> str:

    return PythonObjectEncoder().encode(the_dict)


class SimpleClass:
    def __init__(self):
        self.instance_value = 10


b = SimpleClass()

dict1: dict = {}
dict1["string"] = "Hello, World!"
dict1["datetime"] = datetime(2030, 12, 31)
dict1["nothing"] = None
dict1["set"]: set = {0, 1, 2, 3}
dict1["tuple"] = ("apples", "fresh", "green")
dict1["list"] = ["a", "b", "c"]
dict1["simple_class"] = SimpleClass()

assert (
        dict_to_json(dict1) ==
            (
                '{"string": "Hello, World!",' 
                ' "datetime": "2030-12-31T00:00:00",'
                ' "nothing": null,'
                ' "set": [0, 1, 2, 3],'
                ' "tuple": ["apples", "fresh", "green"],'
                ' "list": ["a", "b", "c"],'
                ' "simple_class": {"instance_value": 10}}'
            )
    )

# 16. Delete Multiple Items from a Dictionary During Iteration

In [17]:
def remove_even_items(the_dict: dict):

    # collect the keys of all the items to remove
    delete_these = set(k for k, v in the_dict.items() if v % 2 == 0)

    for delete_this in delete_these:
        del the_dict[delete_this]

    return the_dict


dict1: dict = dict(enumerate(range(13)))
assert remove_even_items(dict1) == {1: 1, 3: 3, 5: 5, 7: 7, 9: 9, 11: 11}